In [5]:
import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
from PIL import Image
import pathlib
import csv

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import warnings
warnings.filterwarnings('ignore')
#Заголовок csv файла 
header = 'chroma_stft rmse spectral_centroid spectral_bandwidth rolloff zero_crossing_rate'
for i in range(1, 21):
    header += f' mfcc{i}'
header += ' class'
header = header.split()
#Создаем csv файл для записи
file = open('dataset.csv', 'w', newline='')
#Записываем заголовок в файл
with file:
    writer = csv.writer(file)
    writer.writerow(header)
#Пробегаем по всем папкам от 1 до 5 - классам аудиозаписям
for cl in range(1,6):
    #Для всех аудиозаписей в текущей папке
    for filename in os.listdir(f'./Data/{cl}'):
        songname = f'./Data/{cl}/{filename}'
        #Загружаем данные файла waw
        y, sr = librosa.load(songname)
        #Вычисляем среднеквадратичную ошибку трансформации
        rmse = librosa.feature.rms(y=y)[0]
        #вычисляем цветность - высоту тона
        chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
        #вычисляем спектральный центроид - где расположен "центр масс" звука
        spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
        #спектральная ширина
        spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
        #частоты спектрального спада
        rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
        #число пересечений нуля
        zcr = librosa.feature.zero_crossing_rate(y)
        #Мел-частотные кепстральные коэффициенты
        mfcc = librosa.feature.mfcc(y=y, sr=sr)
        #записываем средние значения характеристик
        to_append = f'{np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'    
        for e in mfcc:
            to_append += f' {np.mean(e)}'
        to_append += f' {cl}'
        file = open('dataset.csv', 'a', newline='')
        with file:
            writer = csv.writer(file)
            writer.writerow(to_append.split())